In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

# ----------------------------------
# Load trained models
# ----------------------------------
model_paths = {
    "InceptionV3": "/kaggle/input/inception/kvasir_inceptionv3_model.keras",
    "ResNet18": "/kaggle/working/resnet18_best.keras",
    "MobileNetV2": "/kaggle/working/mobilenetv2_best.keras"
}

models = []
model_names = []

for name, path in model_paths.items():
    try:
        model = tf.keras.models.load_model(path)
        models.append(model)
        model_names.append(name)
        print(f"✅ Loaded {name} from {path}")
    except Exception as e:
        print(f"❌ Could not load {name}: {e}")

num_models = len(models)
assert num_models >= 2, "At least two models are required for ensemble learning."

print(f"\n🔗 Ensemble Models Used: {model_names}")

# ----------------------------------
# Test data preparation
# ----------------------------------
test_dir = "/kaggle/input/testing/Test"
img_size = (224, 224)
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# ----------------------------------
# Ensemble prediction (Softmax Averaging)
# ----------------------------------
ensemble_preds = np.zeros((len(y_true), len(class_labels)))

for model in models:
    preds = model.predict(test_generator, verbose=1)
    ensemble_preds += preds

ensemble_preds /= num_models
y_pred = np.argmax(ensemble_preds, axis=1)

# ----------------------------------
# Evaluation
# ----------------------------------
ensemble_accuracy = accuracy_score(y_true, y_pred)
print(f"\n✅ Ensemble Accuracy ({num_models} models): {ensemble_accuracy * 100:.2f}%")

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# ----------------------------------
# Confusion Matrix
# ----------------------------------
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation="nearest")
plt.title("Ensemble Confusion Matrix")
plt.colorbar()

tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)

for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        plt.text(j, i, cm[i, j], ha="center", va="center")

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()
